In [0]:
from pyspark.sql.functions import col, isnull, when
from pyspark.sql.types import TimestampType
from datetime import date, timedelta

In [0]:
from datetime import date, timedelta

# Remove this before running Data Factory Pipeline
start_date = date.today() - timedelta(1)

bronze_adls = "abfss://bronze@deprojectearthquake.dfs.core.windows.net/"
silver_adls = "abfss://silver@deprojectearthquake.dfs.core.windows.net/"

In [0]:
# Load the JSON data into a Spark DataFrame
df = spark.read.option("multiline", "true").json(f"{bronze_adls}{start_date}_earthquake_data.json")

In [0]:
df

DataFrame[geometry: struct<coordinates:array<double>,type:string>, id: string, properties: struct<alert:string,cdi:double,code:string,detail:string,dmin:double,felt:bigint,gap:double,ids:string,mag:double,magType:string,mmi:double,net:string,nst:bigint,place:string,rms:double,sig:bigint,sources:string,status:string,time:bigint,title:string,tsunami:bigint,type:string,types:string,tz:string,updated:bigint,url:string>, type: string]

In [0]:
df.head()

Row(geometry=Row(coordinates=[-152.295, 61.2895, -0.12], type='Point'), id='av93734213', properties=Row(alert=None, cdi=None, code='93734213', detail='https://earthquake.usgs.gov/fdsnws/event/1/query?eventid=av93734213&format=geojson', dmin=0.03733, felt=None, gap=133.0, ids=',av93734213,', mag=0.64, magType='ml', mmi=None, net='av', nst=9, place='67 km WNW of Tyonek, Alaska', rms=0.06, sig=6, sources=',av,', status='reviewed', time=1756165643860, title='M 0.6 - 67 km WNW of Tyonek, Alaska', tsunami=0, type='earthquake', types=',origin,phase-data,', tz=None, updated=1756182644700, url='https://earthquake.usgs.gov/earthquakes/eventpage/av93734213'), type='Feature')

In [0]:
# Reshape earthquake data
df = (
    df
    .select(
        'id',
        col('geometry.coordinates').getItem(0).alias('longitude'),
        col('geometry.coordinates').getItem(1).alias('latitude'),
        col('geometry.coordinates').getItem(2).alias('elevation'),
        col('properties.title').alias('title'),
        col('properties.place').alias('place_description'),
        col('properties.sig').alias('sig'),
        col('properties.mag').alias('mag'),
        col('properties.magType').alias('magType'),
        col('properties.time').alias('time'),
        col('properties.updated').alias('updated')
    )
)

In [0]:
df.head()

Row(id='av93734213', longitude=-152.295, latitude=61.2895, elevation=-0.12, title='M 0.6 - 67 km WNW of Tyonek, Alaska', place_description='67 km WNW of Tyonek, Alaska', sig=6, mag=0.64, magType='ml', time=1756165643860, updated=1756182644700)

In [0]:
# Validate data: Check for missing or null values
df = (
    df
    .withColumn('longitude', when(isnull(col('longitude')), 0).otherwise(col('longitude')))
    .withColumn('latitude', when(isnull(col('latitude')), 0).otherwise(col('latitude')))
    .withColumn('time', when(isnull(col('time')), 0).otherwise(col('time')))
)

In [0]:
# Convert 'time' and 'updated' to timestamp from Unix time
df = (
    df
    .withColumn('time', (col('time') / 1000).cast(TimestampType()))
    .withColumn('updated', (col('updated') / 1000).cast(TimestampType()))
)

In [0]:
df.head()

Row(id='av93734213', longitude=-152.295, latitude=61.2895, elevation=-0.12, title='M 0.6 - 67 km WNW of Tyonek, Alaska', place_description='67 km WNW of Tyonek, Alaska', sig=6, mag=0.64, magType='ml', time=datetime.datetime(2025, 8, 25, 23, 47, 23, 860000), updated=datetime.datetime(2025, 8, 26, 4, 30, 44, 700000))

In [0]:
# Save the transformed DataFrame to the Silver container
silver_output_path = f"{silver_adls}earthquake_events_silver/"

In [0]:
# Append DataFrame to Silver container in Parquet format
df.write.mode('append').parquet(silver_output_path)

In [0]:
dbutils.jobs.taskValues.set(key = "silver_output", value = silver_output_path)